In [1]:
#Loading VGG16

In [2]:
from keras.applications import VGG16
# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the Vgg16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

Using TensorFlow backend.


In [3]:
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [4]:
#freezing all layers except the top layers

In [13]:
from keras.applications import VGG16
# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the Vgg16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows,img_cols, 3))
#making the default layers from trainable to false
for layers in model.layers:
    layer.trainable = False
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)  

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [6]:
def addTopModel(bottom_model, num_classes,D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    #top_model = GlobalAveragePooling2D()(top_model)
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(D,activation='relu')(top_model)
    #top_model = Dense(1024,activation='relu')(top_model)
    #top_model = Dense(512,activation='relu')(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model


num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs = model.input, outputs = FC_Head)

print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'D:\\New folder\\task4\\Train'
validation_data_dir = 'D:\\New folder\\task4\\test'


#  using data augmentaiton 

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
train_batchsize = 4
val_batchsize = 4
img_rows=224
img_cols=224
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize ,
        class_mode='categorical') #binary    
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize ,
        class_mode= 'categorical',
        shuffle=False)

Found 200 images belonging to 2 classes.
Found 48 images belonging to 2 classes.


In [9]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("task4_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

#earlystop = EarlyStopping(monitor = 'val_loss', 
                          #min_delta = 0, 
                          #patience = 3,
                          #verbose = 1,
                          #restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [checkpoint]

# We use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 200
nb_validation_samples = 48

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("task4_vgg.h5")

Epoch 1/5
12/12 [==============================] - 56s 5s/step - loss: 881.4216 - accuracy: 0.4792 - val_loss: 0.0176 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.01763, saving model to task4_vgg.h5
Epoch 2/5
12/12 [==============================] - 55s 5s/step - loss: 1.0390 - accuracy: 0.4583 - val_loss: 0.6808 - val_accuracy: 1.0000

Epoch 00002: val_loss did not improve from 0.01763
Epoch 3/5
12/12 [==============================] - 54s 5s/step - loss: 0.7590 - accuracy: 0.4375 - val_loss: 0.7980 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss did not improve from 0.01763
Epoch 4/5
12/12 [==============================] - 54s 4s/step - loss: 0.7296 - accuracy: 0.5208 - val_loss: 0.6911 - val_accuracy: 0.8333

Epoch 00004: val_loss did not improve from 0.01763
Epoch 5/5
12/12 [==============================] - 55s 5s/step - loss: 0.6935 - accuracy: 0.6458 - val_loss: 0.7402 - val_accuracy: 0.0000e+00

Epoch 00005: val_loss did not improve from 0.01763


In [10]:
from keras.models import load_model
classifier = load_model("task4_vgg.h5")

In [11]:
train_generator.class_indices

{'didi': 0, 'me': 1}

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile,join

face_dict = {"[0]": "didi",
            "[1]": "me"}

face_dict_n = {"didi":"didi",
            "me":"me"}

def draw_test(name,predt,im):
    face=face_dict[str(predt)]
    BLACK = [0,0,0]
    expand = cv2.copyMakeBorder(im,40,40,40,40,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expand,face,(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.imshow(name,expand)
    
def getRandomImage(path):
        #laoding random sample
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path,x)),os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class -" + face_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path + "/" + image_name)
    
for i in range(0,10):
    input_im = getRandomImage("D:\\New folder\\task4\\test\\,D:\\New folder\\ta,)
    input_original = input_im.copy()
    input_original = cv2.resize(input_original,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im,(224,224),interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255
    input_im = input_im.reshape(1,224,224,3)
    
    res = np.argmax(classifier.predict(input_im,1,verbose = 0),axis=1)
    draw_test("Prediction",res,input_original)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()    

Class -me
Class -me
Class -didi
Class -didi
